In [1]:
import torch
from torch import nn



## 继承Module类来构造模型
`Module`类是`nn`模块里提供的一个模型构造类，是`所有神经网络模块的基类`，可以继承它来定义模型。

这里定义的`MLP类`重载了`Module类`的`__init__函数`和`forward`函数。它们分别用于`创建模型参数`和`定义前向计算`。

In [2]:
class MLP(nn.Module):
    # 声明带有模型参数的层，这里声明了两个全连接层
    def __init__(self, **kwargs):
        # 调用MLP父类Module的构造函数来进行必要的初始化。这样在构造实例时还可以指定其他函数
        # 参数，如“模型参数的访问、初始化和共享”一节将介绍的模型参数params
        super(MLP, self).__init__(**kwargs)
        self.hidden = nn.Linear(784, 256) # 隐藏层
        self.act = nn.ReLU()
        self.output = nn.Linear(256, 10)  # 输出层


    # 定义模型的前向计算，即如何根据输入x计算返回所需要的模型输出
    def forward(self, x):
        a = self.act(self.hidden(x))
        return self.output(a)
    
X = torch.rand(2, 784)
net = MLP()
print(net)
net(X)

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[-0.2821,  0.2484,  0.0579, -0.0173, -0.0993, -0.1798, -0.1104,  0.1019,
         -0.1043, -0.1074],
        [-0.1610,  0.2525,  0.0878, -0.1012,  0.0398, -0.2225, -0.1631,  0.1270,
         -0.1356, -0.1494]], grad_fn=<AddmmBackward>)

## Module的子类
`Module类`是一个通用的部件。`PyTorch`还实现了继承自`Module`的可以方便构建模型的类: 如`Sequential`、`ModuleList`和`ModuleDict`等等。

### Sequential类
当模型的`前向计算为简单串联各个层的计算时`，Sequential类可以通过更加简单的方式定义模型。        

这正是Sequential类的目的：它可以`接收一个子模块的有序字典（OrderedDict）`或者`一系列子模块`作为参数来逐一添加Module的实例，而模型的前向计算就是将这些实例按添加的顺序逐一计算。



In [3]:
class MySequential(nn.Module):
    from collections import OrderedDict
    def __init__(self, *args):
        super(MySequential, self).__init__()
        if len(args) == 1 and isinstance(args[0], OrderedDict): # 如果传入的是一个OrderedDict
            for key, module in args[0].items():
                self.add_module(key, module)  # add_module方法会将module添加进self._modules(一个OrderedDict)
        else:  # 传入的是一些Module
            for idx, module in enumerate(args):
                self.add_module(str(idx), module)
    def forward(self, input):
        # self._modules返回一个 OrderedDict，保证会按照成员添加时的顺序遍历成员
        for module in self._modules.values():
            input = module(input)
        return input


net = MySequential(
        nn.Linear(784, 256),
        nn.ReLU(),
        nn.Linear(256, 10), 
        )
print(net)
net(X)

MySequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[-0.1660, -0.2438,  0.2977, -0.1283, -0.2451,  0.0726, -0.0705, -0.1425,
          0.1517,  0.0466],
        [-0.1380, -0.3152,  0.2592, -0.0546, -0.1657,  0.0285, -0.0626, -0.1840,
          0.0167, -0.0616]], grad_fn=<AddmmBackward>)

### ModuleList类
`ModuleList`接收一个子模块的列表作为输入，然后也可以类似`List`那样进行`append和extend`操作:

In [4]:
net = nn.ModuleList([nn.Linear(784, 256), nn.ReLU()])
net.append(nn.Linear(256, 10)) # # 类似List的append操作
print(net[-1])  # 类似List的索引访问
print(net)
# net(torch.zeros(1, 784)) # 会报NotImplementedError

Linear(in_features=256, out_features=10, bias=True)
ModuleList(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


`Sequential`和`ModuleList`都可以进行列表化构造网络。

- **二者的区别:**   
1. `ModuleList`仅仅是一个储存各种模块的列表，这些模块之间**没有联系也没有顺序**（所以不用保证相邻层的输入输出维度匹配）;`Sequential`内的模块需要按照顺序排列，要**保证相邻层的输入输出大小相匹配**


2. `ModuleList`**没有实现forward功能需要自己实现**，所以上面执行net(torch.zeros(1, 784))会报NotImplementedError；`Sequential`内部forward功能已经实现。



In [5]:
class MyModule(nn.Module):
    def __init__(self):
        super(MyModule, self).__init__()
        self.linears = nn.ModuleList([nn.Linear(10, 10) for i in range(10)])

    def forward(self, x):
        # ModuleList can act as an iterable, or be indexed using ints
        for i, l in enumerate(self.linears):
            x = self.linears[i // 2](x) + l(x)
        return x

net = MyModule()
print(net)
X = torch.ones(10,10)

MyModule(
  (linears): ModuleList(
    (0): Linear(in_features=10, out_features=10, bias=True)
    (1): Linear(in_features=10, out_features=10, bias=True)
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): Linear(in_features=10, out_features=10, bias=True)
    (4): Linear(in_features=10, out_features=10, bias=True)
    (5): Linear(in_features=10, out_features=10, bias=True)
    (6): Linear(in_features=10, out_features=10, bias=True)
    (7): Linear(in_features=10, out_features=10, bias=True)
    (8): Linear(in_features=10, out_features=10, bias=True)
    (9): Linear(in_features=10, out_features=10, bias=True)
  )
)


`ModuleList`不同于一般的`Python`的`list`，加入到`ModuleList`里面的所有模块的参数会被自动添加到整个网络中    

- net1有$w,b$
- net2没有参数

In [6]:
class Module_ModuleList(nn.Module):
    def __init__(self):
        super(Module_ModuleList, self).__init__()
        self.linears = nn.ModuleList([nn.Linear(10, 10)])

class Module_List(nn.Module):
    def __init__(self):
        super(Module_List, self).__init__()
        self.linears = [nn.Linear(10, 10)]

net1 = Module_ModuleList()
net2 = Module_List()

print("net1:")
for p in net1.parameters():
    print(p.size())

print("net2:")
for p in net2.parameters():
    print(p)

net1:
torch.Size([10, 10])
torch.Size([10])
net2:


### ModuleDict类
`ModuleDict`接收一个子模块的字典作为输入, 然后也可以`类似字典`进行添加访问操作:   

和`ModuleList`一样，`ModuleDict`实例仅仅是存放了一些模块的字典，并没有定义`forward`函数。`ModuleDict`里的所有模块的参数会被自动添加到整个网络中。

In [7]:
net = nn.ModuleDict({
    'linear': nn.Linear(784, 256),
    'act': nn.ReLU(),
})
net['output'] = nn.Linear(256, 10) # 添加
print(net['linear']) # 访问
print(net.output)
print(net)
# net(torch.zeros(1, 784)) # 会报NotImplementedError



Linear(in_features=784, out_features=256, bias=True)
Linear(in_features=256, out_features=10, bias=True)
ModuleDict(
  (linear): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


## 构造复杂的模型    

虽然上面这些类可以使模型构造更加简单，且不需要定义forward函数，但**直接继承`Module类`可以极大地拓展模型构造的灵活性**。      

下面构造一个稍微复杂点的网络`FancyMLP`。在这个网络中，我们通过`get_constant`函数创建训练中不被迭代的参数，即常数参数。  

在前向计算中，除了使用创建的常数参数外，使用Tensor的函数和Python的控制流，**并多次调用相同的层**。



In [8]:
class FancyMLP(nn.Module):
    def __init__(self, **kwargs):
        super(FancyMLP, self).__init__(**kwargs)

        self.rand_weight = torch.rand((20, 20), requires_grad=False) # 不可训练参数（常数参数）
        self.linear = nn.Linear(20, 20)

    def forward(self, x):
        x = self.linear(x)
        # 使用创建的常数参数，以及nn.functional中的relu函数和mm函数
        x = nn.functional.relu(torch.mm(x, self.rand_weight.data) + 1)

        # 复用全连接层。等价于两个全连接层共享参数
        x = self.linear(x)
        # 控制流，这里我们需要调用item函数来返回标量进行比较
        while x.norm().item() > 1:
            x /= 2
        if x.norm().item() < 0.8:
            x *= 10
        return x.sum()

# 使用了常数权重rand_weight（注意它不是可训练模型参数）、做了矩阵乘法操作（torch.mm）
# 重复使用了相同的Linear层。

X = torch.rand(2, 20)
net = FancyMLP()
print(net)
net(X)




FancyMLP(
  (linear): Linear(in_features=20, out_features=20, bias=True)
)


tensor(-7.4972, grad_fn=<SumBackward0>)

因为`FancyMLP`和`Sequential类`都是`Module类`的子类，所以我们可以嵌套调用它们。


In [9]:
class NestMLP(nn.Module):
    def __init__(self, **kwargs):
        super(NestMLP, self).__init__(**kwargs)
        self.net = nn.Sequential(nn.Linear(40, 30), nn.ReLU()) 

    def forward(self, x):
        return self.net(x)

net = nn.Sequential(NestMLP(), nn.Linear(30, 20), FancyMLP())

X = torch.rand(2, 40)
print(net)
net(X)

Sequential(
  (0): NestMLP(
    (net): Sequential(
      (0): Linear(in_features=40, out_features=30, bias=True)
      (1): ReLU()
    )
  )
  (1): Linear(in_features=30, out_features=20, bias=True)
  (2): FancyMLP(
    (linear): Linear(in_features=20, out_features=20, bias=True)
  )
)


tensor(-5.4901, grad_fn=<SumBackward0>)